In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import math # Import math for sqrt


cap = cv2.VideoCapture("/content/Australian racing greyhounds  - Dog race [gEX787opZvE].webm") # Use your video path
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("pixel_speed_estimation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

model = YOLO("/content/best (3).pt") # Use your model path

track_history = defaultdict(lambda: [])
frame_count = 0

# Calculate time interval only once
if fps > 0:
    time_interval = 1.0 / fps
else:
    time_interval = 0 # Avoid division by zero, though speed calc will fail
    print("WARNING: Video FPS is 0. Cannot calculate speed.")


# Dictionary to map track_id to simple dog label
dog_label_map = {}
next_dog_number = 1

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_count += 1
    annotated_frame = frame.copy() # Work on a copy

    results = model.track(frame, persist=True, tracker="bytetrack.yaml")

    if results[0].boxes is not None and results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)

        for box, track_id in zip(boxes, track_ids):
            x1, y1, x2, y2 = box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2 # Or use bottom center: y2

            # Assign simple dog label
            if track_id not in dog_label_map:
                dog_label_map[track_id] = f"dog_{next_dog_number}"
                next_dog_number += 1
            dog_label = dog_label_map[track_id]

            # Store history
            current_pos_pixel = (center_x, center_y)
            track = track_history[track_id]
            track.append((frame_count, center_x, center_y))
            if len(track) > 30: track.pop(0)

            pixel_speed = 0.0 # Default speed
            if len(track) > 1 and time_interval > 0:
                # Get previous pixel position
                _, x_prev, y_prev = track[-2]
                prev_pos_pixel = (x_prev, y_prev)

                # Calculate pixel distance
                pixel_distance = math.sqrt(
                    (current_pos_pixel[0] - prev_pos_pixel[0])**2 +
                    (current_pos_pixel[1] - prev_pos_pixel[1])**2
                )

                # Calculate pixel speed (pixels per second)
                pixel_speed = pixel_distance / time_interval

            # --- Manual Annotation ---
            color = (0, 255, 0)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

            # Format text label with pixel speed
            label_text = f"{dog_label}: {pixel_speed:.1f} pixels/s" # Changed unit

            (w_text, h_text), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > h_text else y1 + h_text + 10

            cv2.rectangle(annotated_frame, (text_x, text_y - h_text - 2), (text_x + w_text, text_y + 2), color, -1)
            cv2.putText(annotated_frame, label_text, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            # -------------------------

        video_writer.write(annotated_frame)
    else:
        video_writer.write(frame)


cap.release()
video_writer.release()
cv2.destroyAllWindows()

print("Processing complete. Output saved to pixel_speed_estimation.avi")